# Tensorboard 2

This is a set of cells that gather examples from:
- Learn Tensorflow 2.0, Singh & Manure, 2019
- Tensorboard What's new in TensorFlow's visualization toolkit, TensorFlow Dev Summit, March, 2019

### Example 1

Basic example: doing a Sequential model, autogenerating the data base on np.random.sample.

In [1]:
%load_ext tensorboard
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
from datetime import datetime
from tensorflow import keras
print(tf.__version__)

2.0.0-beta1


In [2]:
# Creating data
X_train = (np.random.sample((10000,5)))
y_train =  (np.random.sample((10000,1)))
X_train.shape
print(X_train)

[[0.73951549 0.35126237 0.43514608 0.59635915 0.26008277]
 [0.08427734 0.31638969 0.73797794 0.71426958 0.51056576]
 [0.89014269 0.40005075 0.35648198 0.04393161 0.42869708]
 ...
 [0.36070357 0.08176955 0.07583313 0.7109154  0.21631404]
 [0.89167466 0.45903446 0.71095811 0.42431557 0.47153313]
 [0.95396516 0.45590081 0.23032739 0.87325127 0.36933122]]


In [3]:
# Build the tf.keras.Sequential model by stacking layers.
model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [4]:
# Train the estimator
model.fit(X_train, y_train, epochs=10, callbacks=[tensorboard_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 2s 213us/sample - loss: 0.0674 - accuracy: 0.0000e+00
Epoch 2/10
10000/10000 [==============================] - 1s 56us/sample - loss: 7.8399e-06 - accuracy: 0.0000e+00
Epoch 3/10
10000/10000 [==============================] - 1s 56us/sample - loss: 2.8972e-06 - accuracy: 0.0000e+00
Epoch 4/10
10000/10000 [==============================] - 1s 57us/sample - loss: 1.5978e-06 - accuracy: 0.0000e+00
Epoch 5/10
10000/10000 [==============================] - 1s 56us/sample - loss: 1.5851e-06 - accuracy: 0.0000e+00
Epoch 6/10
10000/10000 [==============================] - 1s 56us/sample - loss: 1.5761e-06 - accuracy: 0.0000e+00
Epoch 7/10
10000/10000 [==============================] - 1s 59us/sample - loss: 1.1278e-06 - accuracy: 0.0000e+00
Epoch 8/10
10000/10000 [==============================] - 1s 62us/sample - loss: 

In [5]:
model.evaluate(X_train, y_train)

10000/10000 [==============================] - 0s 21us/sample - loss: 9.9965e-07 - accuracy: 0.0000e+00


[9.99648073593562e-07, 0.0]

In [18]:
%tensorboard --logdir logs/fit

## Example 2

Sequential model using MNIST dataset, training with Fit API.

In [ ]:
from tensorflow.keras.datasets import fashion_mnist


In [7]:
from tensorflow.keras.datasets import mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()

In [8]:
img_rows, img_cols = 28, 28

#if keras.backend.image_data_format() == 'channels_first':
 #   shape_ord = (1, img_rows, img_cols)
#else:  # channel_last
 #   shape_ord = (img_rows, img_cols, 1)

#Normalizing data
#X_train = X_train.reshape((X_train.shape[0],) + shape_ord)
#X_test = X_test.reshape((X_test.shape[0],) + shape_ord)

X_train = x_train.astype('float32')
X_test = x_test.astype('float32')

X_train /= 255
X_test /= 255

In [9]:
def train_test_model(run_dir,hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams['num_units'], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams['dropout_rate']),
        tf.keras.layers.Dense(10,activation=tf.nn.softmax),
    ])
    
    model.compile(optimizer=hparams['optimizer'],
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    model.fit(X_train,y_train,
             validation_data=(X_test,y_test),
             epochs=3,
             callbacks=[tf.keras.callbacks.TensorBoard(run_dir + "/keras")])
    
    scores = model.evaluate(X_test,y_test)
    return scores

In [11]:
train_test_model('logs/sample',{'num_units':32, 'dropout_rate': 0.2, 'optimizer': 'adam'})

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4575 - accuracy: 0.8669 - val_loss: 0.2218 - val_accuracy: 0.9382
Epoch 2/3
60000/60000 [==============================] - 2s 40us/sample - loss: 0.2654 - accuracy: 0.9214 - val_loss: 0.1674 - val_accuracy: 0.9494
Epoch 3/3
10000/10000 [==============================] - 0s 17us/sample - loss: 0.1468 - accuracy: 0.9568


[0.14675281562581657, 0.9568]

In [13]:
%tensorboard --logdir logs/sample/

## Example 3

The same example 2 but using Tensorboard plugins for hyper parameters tuning and some new stuff for experimental summary configuration.

In [14]:
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2
from tensorboard.plugins.hparams import api

In [15]:
num_units_list =[16,32]
dropout_rate_list= [0.1, 0.2]
optimizer_list = ['adam']

In [16]:
def create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list):
    
    num_units_list_val = struct_pb2.ListValue()
    num_units_list_val.extend(num_units_list)
    dropout_rate_list_val = struct_pb2.ListValue()
    dropout_rate_list_val.extend(dropout_rate_list)
    optimizer_list_val = struct_pb2.ListValue()
    optimizer_list_val.extend(optimizer_list)
    return hparams_summary.experiment_pb(
        #List our hyperparameters
        hparam_infos=[
            api_pb2.HParamInfo(name = 'num_units',
                              display_name = '# Units',
                              type = api_pb2.DATA_TYPE_FLOAT64,
                              domain_discrete=num_units_list_val),
            api_pb2.HParamInfo(name = 'dropout_rate',
                              display_name = 'Dropout Rate',
                              type = api_pb2.DATA_TYPE_FLOAT64,
                              domain_discrete=dropout_rate_list_val),
            api_pb2.HParamInfo(name = 'optimizer',
                              display_name = 'Optimizer',
                              type = api_pb2.DATA_TYPE_STRING,
                              domain_discrete=optimizer_list_val)
        ],
        #List our metrics
        metric_infos=[
            api_pb2.MetricInfo(
                name=api_pb2.MetricName(
                    tag = 'accuracy'),
                display_name='Accuracy'),
        ]
    )

exp_summary = create_experiment_summary(num_units_list, dropout_rate_list, optimizer_list)
logdir_writer = tf.compat.v2.summary.create_file_writer('logs/hparam_tuning')
with logdir_writer.as_default():
    tf.compat.v2.summary.experimental.write_raw_pb(exp_summary.SerializeToString(), step=0)

In [17]:
def run(run_dir, hparams):
    writer = tf.summary.create_file_writer(run_dir)
    summary_start = hparams_summary.session_start_pb(hparams=hparams)
    
    with writer.as_default():
        tf.compat.v2.summary.experimental.write_raw_pb(exp_summary.SerializeToString(), step=0)
        loss, accuracy = train_test_model(run_dir, hparams)
        
        tf.summary.scalar('accuracy', accuracy, step=0, description='The Accuracy')
        summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)
        tf.compat.v2.summary.experimental.write_raw_pb(summary_end.SerializeToString(), step=0)

In [19]:
%tensorboard --logdir logs/hparam_tuning

In [71]:
session_num = 0

for num_units in num_units_list:
    for dropout_rate in dropout_rate_list:
        for optimizer in optimizer_list:
            hparams = {'num_units': num_units, 'dropout_rate': dropout_rate, 'optimizer': optimizer}
            print('---- Running training session %d' % (session_num+1))
            print(hparams)
            run_name = "run-%d" % session_num
            run('logs/hparam_tuning/'+run_name, hparams)
            session_num +=1

---- Running training session 1
{'num_units': 16, 'dropout_rate': 0.1, 'optimizer': 'adam'}
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 5s 87us/sample - loss: 0.5698 - accuracy: 0.8277 - val_loss: 0.2793 - val_accuracy: 0.9212
Epoch 2/3
60000/60000 [==============================] - 3s 54us/sample - loss: 0.3520 - accuracy: 0.8931 - val_loss: 0.2348 - val_accuracy: 0.9310
Epoch 3/3
10000/10000 [==============================] - 0s 25us/sample - loss: 0.2130 - accuracy: 0.9360
---- Running training session 2
{'num_units': 16, 'dropout_rate': 0.2, 'optimizer': 'adam'}
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 4s 61us/sample - loss: 0.6427 - accuracy: 0.8004 - val_loss: 0.2946 - val_accuracy: 0.9171
Epoch 2/3
60000/60000 [==============================] - 4s 62us/sample - loss: 0.4241 - accuracy: 0.8688 - val_loss: 0.2633 - val_accuracy: 0.9241
Epoch 3/3
10000/10

## Conclussions

The codes for tensorboard are complex, the configuration for hyperparameter tunning is very useful but must be studied in deep.

The examples given in March of 2019 were deprecated in April due to the fast rate of changes of tensorboard library 2.x.

This refactorization of March 2019 codes has some problems because isn't show the the rows in "Table View", therefore the graphics in "Parallel Cordinate Views" are not shown either.

### Recomendation

* It is a good practice to follow the changes in github, some practical examples, when you do diff, are shown graphically, so you can see the old version and how to implement the same example in the new version.

* The recommended url is [https://github.com/tensorflow/tensorboard](https://github.com/tensorflow/tensorboard.git)

* ESPECIAL recommendation to the jupyter notebook [https://github.com/tensorflow/tensorboard/docs/hyperparameter_tuning_with_hparams](https://github.com/tensorflow/tensorboard/blob/master/docs/hyperparameter_tuning_with_hparams.ipynb)